# 💬 TRM POC - Dialogue avec Spinoza (Modèle SPS Fine-tuné)

**Modèle:** `FJDaz/spinoza-mistral-7b-merged` (votre modèle fine-tuné !)

**Runtime:** GPU Colab gratuit (T4 - 15GB VRAM)

---

## 🎯 Différence avec Mistral de base

✅ **Modèle SPS** (FJDaz/spinoza-mistral-7b-merged) :
- Fine-tuné spécifiquement sur corpus Spinoza
- Connaît déjà conatus, affects, Éthique
- Meilleure qualité dialogue philosophique

❌ Mistral 7B Instruct (base) :
- Généraliste, pas spécialisé philo
- Nécessite plus de contexte RAG

## 1. Installation Dépendances

In [ ]:
!pip install -q transformers torch sentencepiece spacy accelerate bitsandbytes
!pip install -q sentence-transformers faiss-gpu gradio
!python -m spacy download fr_core_news_sm

print("✅ Dépendances installées")

## 2. Choix Source Modèle SPS

**2 options pour charger votre modèle SPS :**

In [ ]:
# ============================================
# CONFIGURATION MODÈLE SPS
# ============================================

# Choisir UNE des 2 options ci-dessous :

# --- OPTION A : HuggingFace (Plus simple, mais plus lent 1ère fois) ---
USE_HUGGINGFACE = True
MODEL_NAME = "FJDaz/spinoza-mistral-7b-merged"

# --- OPTION B : Google Drive (Plus rapide si déjà téléchargé) ---
USE_GOOGLE_DRIVE = False
DRIVE_MODEL_PATH = "/content/drive/MyDrive/spinoza-mistral-7b-merged"

# ============================================

if USE_GOOGLE_DRIVE:
    print("📂 Option B : Chargement depuis Google Drive")
    from google.colab import drive
    drive.mount('/content/drive')
    
    MODEL_PATH = DRIVE_MODEL_PATH
    print(f"✅ Drive monté - Modèle dans: {MODEL_PATH}")
    
elif USE_HUGGINGFACE:
    print("🤗 Option A : Chargement depuis HuggingFace")
    MODEL_PATH = MODEL_NAME
    print(f"✅ Modèle HF: {MODEL_PATH}")
    print("⏳ Note: Téléchargement ~14GB la 1ère fois (5-10 min)")

else:
    raise ValueError("Activer USE_HUGGINGFACE ou USE_GOOGLE_DRIVE")

## 3. Upload RAG Exports

In [ ]:
import os
from google.colab import files

print("📤 Upload rag_exports.zip (depuis Notebook 2)\n")
uploaded = files.upload()

if 'rag_exports.zip' in uploaded:
    !unzip -q rag_exports.zip
    RAG_DIR = "/content/content/rag_exports"
    print("✅ RAG exports chargés")
else:
    print("⚠️ Pas de RAG - Fonctionnera en mode dégradé")
    RAG_DIR = None

## 4. Classes Pipeline (Code réutilisé)

In [ ]:
# ============================================================
# BERT ENCODER
# ============================================================

from transformers import AutoTokenizer, AutoModel
import spacy
import json
from typing import List, Dict, Optional
from collections import Counter

nlp = spacy.load("fr_core_news_sm")

class BERTEncoder:
    def __init__(self, model_name: str = "camembert-base"):
        print(f"⏳ Chargement BERT {model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.eval()
        print("✅ BERT chargé (CPU)")
    
    def extract_keywords(self, text: str, top_k: int = 5) -> List[str]:
        doc = nlp(text)
        entities = [ent.text.lower() for ent in doc.ents]
        nouns = [token.text.lower() for token in doc 
                 if token.pos_ in ["NOUN", "PROPN"] and len(token.text) > 3]
        all_keywords = entities + nouns
        counter = Counter(all_keywords)
        return [word for word, count in counter.most_common(top_k)]
    
    def extract_concepts_from_rag(self, rag_passages: List[Dict]) -> List[str]:
        concepts = []
        for passage in rag_passages:
            if passage.get("concepts"):
                concepts.extend(passage["concepts"][:3])
            else:
                text = passage.get("text", "")
                keywords = self.extract_keywords(text, top_k=3)
                concepts.extend(keywords)
        return list(dict.fromkeys(concepts))[:8]
    
    def analyze_intention(self, text: str) -> str:
        text_lower = text.lower()
        if any(m in text_lower for m in ["?", "comment", "pourquoi"]):
            return "question"
        elif any(m in text_lower for m in ["d'accord", "ok", "oui"]):
            return "accord"
        elif any(m in text_lower for m in ["non", "mais", "faux"]):
            return "désaccord"
        return "neutre"
    
    def encode_to_state_image(
        self,
        conversation: List[Dict],
        rag_passages: List[Dict],
        prev_state: Optional[Dict] = None,
        mini_store_feedback: Optional[Dict] = None
    ) -> Dict:
        last_exchange = conversation[-1] if conversation else {}
        user_text = last_exchange.get("user", "")
        
        return {
            "concepts_actifs": self.extract_keywords(user_text, top_k=5),
            "concepts_rag": self.extract_concepts_from_rag(rag_passages),
            "sources_rag": [p.get("source", "?") for p in rag_passages],
            "intention": self.analyze_intention(user_text),
            "style": "pédagogique",
            "metadata": {
                "philosopher": "spinoza",
                "turn": (prev_state.get("metadata", {}).get("turn", 0) + 1) if prev_state else 1
            }
        }

print("✅ BERTEncoder défini")

In [ ]:
# ============================================================
# RAG RETRIEVER
# ============================================================

from sentence_transformers import SentenceTransformer
import faiss
import pickle

class RAGRetriever:
    def __init__(self, rag_dir: str, philosopher: str = "spinoza"):
        print(f"⏳ Chargement RAG pour {philosopher}...")
        self.embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
        self.philosopher = philosopher
        
        index_path = f"{rag_dir}/{philosopher}_faiss.index"
        self.index = faiss.read_index(index_path)
        
        passages_path = f"{rag_dir}/{philosopher}_passages.pkl"
        with open(passages_path, 'rb') as f:
            self.passages = pickle.load(f)
        
        print(f"✅ RAG chargé: {len(self.passages)} passages")
    
    def retrieve(self, query: str, top_k: int = 3) -> List[Dict]:
        query_emb = self.embedder.encode([query], convert_to_numpy=True)
        faiss.normalize_L2(query_emb)
        
        scores, indices = self.index.search(query_emb, top_k)
        
        results = []
        for score, idx in zip(scores[0], indices[0]):
            if score >= 0.45:
                passage = self.passages[idx].copy()
                passage["similarity_score"] = float(score)
                results.append(passage)
        
        return results[:top_k]

print("✅ RAGRetriever défini")

## 5. Chargement Modèle SPS

In [ ]:
# ============================================================
# SPINOZA MISTRAL GENERATOR (Modèle SPS Fine-tuné)
# ============================================================

import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import time

class SpinozaMistralGenerator:
    """Générateur avec modèle SPS fine-tuné"""
    
    def __init__(self, model_path: str):
        print(f"⏳ Chargement SPS (Spinoza) depuis: {model_path}")
        print("⚠️ Première fois: téléchargement ~14GB (5-10 min)")
        print("⚠️ Chargement + quantization: 5-10 min supplémentaires")
        
        start_time = time.time()
        
        # Tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Configuration quantization 4-bit (pour T4 15GB)
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True
        )
        
        # Charger modèle SPS
        self.model = AutoModelForCausalLM.from_pretrained(
            model_path,
            quantization_config=quant_config,
            device_map="auto",
            trust_remote_code=True,
            token=None  # Pas de token nécessaire pour modèle public
        )
        
        load_time = time.time() - start_time
        print(f"✅ SPS chargé en {load_time:.1f}s")
        
        # Vérifier VRAM
        vram_used = (torch.cuda.mem_get_info()[1] - torch.cuda.mem_get_info()[0]) / 1024**3
        print(f"📊 VRAM utilisée: {vram_used:.2f} GB")
    
    def format_state_image(self, state_image: Dict) -> str:
        lines = []
        if state_image.get("concepts_actifs"):
            lines.append(f"Concepts: {', '.join(state_image['concepts_actifs'][:5])}")
        if state_image.get("concepts_rag"):
            lines.append(f"Corpus: {', '.join(state_image['concepts_rag'][:5])}")
        if state_image.get("intention"):
            lines.append(f"Intention: {state_image['intention']}")
        return "\n".join(lines)
    
    def generate(
        self,
        state_image: Dict,
        user_input: str,
        system_prompt: str,
        max_new_tokens: int = 300
    ) -> str:
        state_text = self.format_state_image(state_image)
        
        prompt = f"""<s>[INST] {system_prompt}

[CONTEXT]
{state_text}

[QUESTION]
{user_input}

Réponds en incarnant Spinoza. [/INST]"""
        
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=self.tokenizer.eos_token_id
        )
        
        response = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:], 
            skip_special_tokens=True
        )
        
        return response

# Charger le modèle SPS
sps_generator = SpinozaMistralGenerator(MODEL_PATH)

print("\n✅ Modèle SPS prêt !")

## 6. Pipeline Complet TRM + SPS

In [ ]:
# Charger composants
USE_BERT = True  # Mettre False si VRAM insuffisante

print("\n🚀 Initialisation Pipeline TRM + SPS")

# RAG
if RAG_DIR and os.path.exists(f"{RAG_DIR}/spinoza_faiss.index"):
    rag = RAGRetriever(RAG_DIR, "spinoza")
else:
    print("⚠️ RAG non disponible")
    rag = None

# BERT (optionnel)
if USE_BERT:
    bert = BERTEncoder()
else:
    bert = None

# Prompt système Spinoza
SYSTEM_PROMPT = """Tu ES Spinoza. Dialogue pédagogique avec élève Terminale.
Vocabulaire: conatus, affects, puissance d'agir, nécessité causale.
Méthode: Déductions logiques, exemples concrets modernes.
Réponds en 2-4 phrases max."""

# Pipeline TRM
class TRMPipeline:
    def __init__(self, rag, bert, sps, system_prompt):
        self.rag = rag
        self.bert = bert
        self.sps = sps
        self.system_prompt = system_prompt
        self.conversation_history = []
        self.state_image = None
    
    def chat(self, user_input: str) -> str:
        # 1. RAG
        rag_passages = self.rag.retrieve(user_input, top_k=3) if self.rag else []
        
        # 2. BERT STATE_IMAGE
        if self.bert:
            self.conversation_history.append({"user": user_input, "assistant": ""})
            self.state_image = self.bert.encode_to_state_image(
                self.conversation_history,
                rag_passages,
                self.state_image,
                {}
            )
        else:
            self.state_image = {
                "concepts_actifs": [],
                "concepts_rag": [c for p in rag_passages for c in p.get("concepts", [])[:2]],
                "intention": "question"
            }
        
        # 3. SPS Generate
        response = self.sps.generate(
            self.state_image,
            user_input,
            self.system_prompt
        )
        
        if self.conversation_history:
            self.conversation_history[-1]["assistant"] = response
        
        return response
    
    def reset(self):
        self.conversation_history = []
        self.state_image = None

# Initialiser pipeline
pipeline = TRMPipeline(rag, bert, sps_generator, SYSTEM_PROMPT)

print("✅ Pipeline TRM + SPS prêt !")

## 7. Test Rapide

In [ ]:
query = "C'est quoi le conatus ?"
print(f"👤 Vous: {query}\n")

response = pipeline.chat(query)
print(f"🧙 Spinoza (SPS): {response}")

## 8. Interface Gradio

In [ ]:
import gradio as gr

def chat_interface(user_input, history):
    if not user_input.strip():
        return history, history
    
    response = pipeline.chat(user_input)
    history.append((user_input, response))
    
    return history, history

def reset_conversation():
    pipeline.reset()
    return [], []

with gr.Blocks(title="Spinoza TRM (SPS Fine-tuné)") as demo:
    gr.Markdown(
        """
        # 🧙 Dialogue avec Spinoza
        
        **Modèle:** FJDaz/spinoza-mistral-7b-merged (fine-tuné)
        
        **Architecture TRM:** RAG + BERT + SPS
        """
    )
    
    chatbot = gr.Chatbot(label="Conversation", height=400)
    
    with gr.Row():
        user_input = gr.Textbox(
            label="Votre question",
            placeholder="Ex: C'est quoi le conatus ?",
            scale=4
        )
        submit_btn = gr.Button("Envoyer", scale=1)
    
    clear_btn = gr.Button("Réinitialiser")
    
    history_state = gr.State([])
    
    submit_btn.click(
        chat_interface,
        inputs=[user_input, history_state],
        outputs=[chatbot, history_state]
    )
    
    user_input.submit(
        chat_interface,
        inputs=[user_input, history_state],
        outputs=[chatbot, history_state]
    )
    
    clear_btn.click(
        reset_conversation,
        outputs=[chatbot, history_state]
    )

demo.launch(share=True, debug=True)

print("\n🚀 Interface Gradio lancée avec modèle SPS !")

---

## ✅ Résumé

### Modèle Utilisé
- **SPS:** `FJDaz/spinoza-mistral-7b-merged` (fine-tuné Spinoza)
- **Avantage:** Meilleure qualité dialogue philo vs Mistral base

### Options Chargement
- **Option A (HF):** Simple, téléchargement auto ~14GB
- **Option B (Drive):** Plus rapide si modèle déjà téléchargé

### Pipeline Complet
- ✅ RAG retrieve passages Éthique
- ✅ BERT génère STATE_IMAGE
- ✅ SPS génère réponse fine-tunée

---

**🎉 Vous dialoguez maintenant avec votre modèle SPS fine-tuné !**